In [20]:
!pip install torch torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())


torch.cuda.empty_cache()

1.3.1
True


In [21]:
!wget https://www.di.ens.fr/willow/teaching/recvis18/assignment3/bird_dataset.zip

--2019-11-26 18:56:43--  https://www.di.ens.fr/willow/teaching/recvis18/assignment3/bird_dataset.zip
Resolving www.di.ens.fr (www.di.ens.fr)... 129.199.99.14
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘bird_dataset.zip.1’

bird_dataset.zip.1      [   <=>              ] 183.48M  30.0MB/s    in 6.2s    

2019-11-26 18:56:49 (29.7 MB/s) - ‘bird_dataset.zip.1’ saved [192388716]



In [0]:
import zipfile
with zipfile.ZipFile("bird_dataset.zip",'r') as zip_ref:
  zip_ref.extractall()

In [0]:
import zipfile
import os

import torchvision.transforms as transforms
#import pretrainedmodels

# once the images are loaded, how do we pre-process them before being passed into the network
# by default, we resize the images to 64 x 64 in size
# and normalize them to mean = 0 and standard-deviation = 1 based on statistics collected from
# the training set
data_train_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomPerspective(),
    transforms.RandomVerticalFlip(),
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
])


data_val_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
])


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

nclasses = 20 

#notre modèle
model = models.resnet152(pretrained= True)

#on freeze
for param in model.parameters():
  param.requires_grad = False


number_features = model.fc.in_features

model.fc = nn.Linear(number_features, 20)


In [0]:
import argparse
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torch.autograd import Variable
from tqdm import tqdm
from torch.optim import lr_scheduler

torch.cuda.empty_cache()

In [26]:
# Training settings
#parser = argparse.ArgumentParser(description='RecVis A3 training script')
#parser.add_argument('--data', type=str, default='bird_dataset', metavar='D',
#                    help="folder where data is located. train_images/ and val_images/ need to be found in the folder")
#parser.add_argument('--batch-size', type=int, default=64, metavar='B',
#                    help='input batch size for training (default: 64)')
#parser.add_argument('--epochs', type=int, default=10, metavar='N',
#                    help='number of epochs to train (default: 10)')
#parser.add_argument('--lr', type=float, default=0.1, metavar='LR',
#                    help='learning rate (default: 0.01)')
#parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
#                    help='SGD momentum (default: 0.5)')
#parser.add_argument('--seed', type=int, default=1, metavar='S',
#                    help='random seed (default: 1)')
#arser.add_argument('--log-interval', type=int, default=10, metavar='N',
#                    help='how many batches to wait before logging training status')
#parser.add_argument('--experiment', type=str, default='experiment', metavar='E',
#                    help='folder where experiment outputs are located.')

#args = parser.parse_args()
use_cuda = torch.cuda.is_available()
#torch.manual_seed(args.seed)

# Create experiment folder
if not os.path.isdir("nouveau"):
    os.makedirs("nouveau")

# Data initialization and loading
#from data import data_transforms

train_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder('bird_dataset' + '/train_images',
                         transform=data_train_transforms),
    batch_size= 32, shuffle=True, num_workers=1)
val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder('bird_dataset' + '/val_images',
                         transform=data_val_transforms),
    batch_size= 32, shuffle=False, num_workers=1)

# Neural network and optimizer
# We define neural net in model.py so that it can be reused by the evaluate.py script
#from model import Net
model = model
if use_cuda:
    print('Using GPU')
    model.cuda()
else:
    print('Using CPU')


optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        criterion = torch.nn.CrossEntropyLoss(reduction='mean')
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))

def validation():
    model.eval()
    validation_loss = 0
    correct = 0
    for data, target in val_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        output = model(data)
        # sum up batch loss
        criterion = torch.nn.CrossEntropyLoss(reduction='mean')
        validation_loss += criterion(output, target).data.item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    validation_loss /= len(val_loader.dataset)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        validation_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))


for epoch in range(1, 50 + 1):
    train(epoch)
    validation()
    model_file = "nouveau" + '/model_' + str(epoch) + '.pth'
    torch.save(model.state_dict(), model_file)
    print('Saved model to ' + model_file + '. You can run `python evaluate.py --model ' + model_file + '` to generate the Kaggle formatted csv file\n')
    

Using GPU
Train Epoch: 1 [0/1082 (0%)]	Loss: 3.090602
Train Epoch: 1 [320/1082 (29%)]	Loss: 3.068284
Train Epoch: 1 [640/1082 (59%)]	Loss: 2.937843
Train Epoch: 1 [960/1082 (88%)]	Loss: 2.904269

Validation set: Average loss: 0.1034, Accuracy: 24/103 (23%)
Saved model to nouveau/model_1.pth. You can run `python evaluate.py --model nouveau/model_1.pth` to generate the Kaggle formatted csv file

Train Epoch: 2 [0/1082 (0%)]	Loss: 2.793746
Train Epoch: 2 [320/1082 (29%)]	Loss: 2.667759
Train Epoch: 2 [640/1082 (59%)]	Loss: 2.516579
Train Epoch: 2 [960/1082 (88%)]	Loss: 2.379550

Validation set: Average loss: 0.0889, Accuracy: 45/103 (44%)
Saved model to nouveau/model_2.pth. You can run `python evaluate.py --model nouveau/model_2.pth` to generate the Kaggle formatted csv file

Train Epoch: 3 [0/1082 (0%)]	Loss: 2.456465
Train Epoch: 3 [320/1082 (29%)]	Loss: 2.419438
Train Epoch: 3 [640/1082 (59%)]	Loss: 2.190546
Train Epoch: 3 [960/1082 (88%)]	Loss: 2.351699

Validation set: Average loss: 

In [0]:
import argparse
from tqdm import tqdm
import os
import PIL.Image as Image

import torch

parser = argparse.ArgumentParser(description='RecVis A3 evaluation script')
parser.add_argument('--data', type=str, default='bird_dataset', metavar='D',
                    help="folder where data is located. test_images/ need to be found in the folder")
parser.add_argument('--model', type=str, metavar='M',
                    help="the model file to be evaluated. Usually it is of the form model_X.pth")
parser.add_argument('--outfile', type=str, default='experiment/kaggle.csv', metavar='D',
                    help="name of the output csv file")

#args = parser.parse_args()
use_cuda = torch.cuda.is_available()

state_dict = torch.load("/content/nouveau/model_30.pth")
model = model
model.load_state_dict(state_dict)
model.eval()
if use_cuda:
    print('Using GPU')
    model.cuda()
else:
    print('Using CPU')

#from data import data_transforms

test_dir = 'bird_dataset' + '/test_images/mistery_category'

def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')


output_file = open('kaggle.csv', "w")
output_file.write("Id,Category\n")
for f in tqdm(os.listdir(test_dir)):
    if 'jpg' in f:
        data = data_val_transforms(pil_loader(test_dir + '/' + f))
        data = data.view(1, data.size(0), data.size(1), data.size(2))
        if use_cuda:
            data = data.cuda()
        output = model(data)
        pred = output.data.max(1, keepdim=True)[1]
        output_file.write("%s,%d\n" % (f[:-4], pred))

output_file.close()

print("Succesfully wrote " + ', you can upload this file to the kaggle competition website')
        

  1%|          | 4/517 [00:00<00:16, 31.52it/s]

Using GPU


100%|██████████| 517/517 [00:14<00:00, 35.33it/s]

Succesfully wrote , you can upload this file to the kaggle competition website
